In [1]:
#include <iostream>
#include <string>
#include <math.h>       /* cosh, log */
using namespace std;

In [2]:
/*Cargo la librería de Delphes*/
gSystem->Load("/home/camilo/Proyectos/Delphes/delphes/libDelphes.so");

Error in <TCling::RegisterModule>: cannot find dictionary module libClassesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libDisplayDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libFastJetDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libModulesDict_rdict.pcm
Error in <TCling::RegisterModule>: cannot find dictionary module libExRootAnalysisDict_rdict.pcm


In [3]:
/*Instancio un objeto TChain*/
TChain chain("Delphes");

In [4]:
/*Creo las variables necesarias para el for*/
std::string nom1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/Signal_2j_mu-nu/Events/run_";
std::string nom2="/tag_1_delphes_events.root";
std::string num, out;

In [5]:
/*Llenado del chain con todos los runs*/
for ( i=1; i<11; i=i+1 ) {
   num = std::to_string(i);
   if( i < 10 ) {
       out= nom1+"0"+num+nom2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * c = out.c_str();
/*pego todos los datos*/
    chain.Add(c);
}

In [6]:
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReader = new ExRootTreeReader(&chain);

In [7]:
/*Leo el numero de entradas*/
Long64_t numberOfEntries = treeReader->GetEntries()

(long long) 354966


In [8]:
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuon = treeReader->UseBranch("Muon");
TClonesArray *branchMissingET = treeReader->UseBranch("MissingET");
TClonesArray *branchJet = treeReader->UseBranch("Jet");

In [9]:
// Inicializo los contadores
Int_t cut1=0, cut2=0, cut3=0, cut4=0, cut5=0;

In [10]:
for(Int_t entry = 0; entry < numberOfEntries; ++entry)
    {
        // Se carga un evento espesifico para ser analizado
        treeReader->ReadEntry(entry);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJet->GetEntries() > 1)
        {
            ++cut1; //Cuantos eventos pasan el corte 1
            
            Int_t count=0;
            //Se cuentan los jets tageados como b
            for (Int_t i=0; i<branchJet->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJet->At(i);
                if(jet->BTag>0)
                {
                    ++count;
                }
    
            }

            if( count>1 )
            {
                ++cut2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1, *muon2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuon->GetEntries() > 1)
                {
                    ++cut3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1 = (Muon *) branchMuon->At(0);
                    muon2 = (Muon *) branchMuon->At(1);
                    //Si la carga es diferente
                    int charge = muon1->Charge * muon2->Charge;
                    if( charge < 0 )
                    {
                        ++cut4; //Cuantos eventos pasan el corte 4
                
                        double MassMuon=(muon1->P4()+muon2->P4()).M();
                        if( !((MassMuon>80)&&(MassMuon<100)) )
                        {
                            ++cut5; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }


Thread 9 (Thread 0x7f772bfff700 (LWP 2337)):
#0  0x00007f7755d39e4b in __GI___waitpid (pid=2340, stat_loc=stat_loc
entry=0x7f772bffb510, options=options
entry=0) at ../sysdeps/unix/sysv/linux/waitpid.c:29
#1  0x00007f7755cc108b in do_system (line=<optimized out>) at ../sysdeps/posix/system.c:148
#2  0x00007f775454ff92 in TUnixSystem::Exec (shellcmd=<optimized out>, this=0x5591aece5e00) at /home/camilo/Proyectos/Root/root/core/unix/src/TUnixSystem.cxx:2118
#3  TUnixSystem::StackTrace (this=0x5591aece5e00) at /home/camilo/Proyectos/Root/root/core/unix/src/TUnixSystem.cxx:2405
#4  0x00007f774d5735d5 in cling::MultiplexInterpreterCallbacks::PrintStackTrace() () from /home/camilo/Proyectos/Root/root/compile/lib/libCling.so
#5  0x00007f774d572f78 in cling_runtime_internal_throwIfInvalidPointer () from /home/camilo/Proyectos/Root/root/compile/lib/libCling.so
#6  0x00007f7756b8d2ad in ?? ()
#7  0x00005591af403990 in ?? ()
#8  0x00007f772bffd7d0 in ?? ()
#9  0x00007f772492e990 in ?? ()
#10 0x0

In [ ]:
cout<<"#Raw ="<<numberOfEntries<<endl;
cout<<"#Cut1="<<cut1<<endl;
cout<<"#Cut2="<<cut2<<endl;
cout<<"#Cut3="<<cut3<<endl;
cout<<"#Cut4="<<cut4<<endl;
cout<<"#Cut5="<<cut5<<endl<<endl;


float EFFN1=(float)cut1/(float)numberOfEntries;
float EFFN2=(float)cut2/(float)cut1;
float EFFN3=(float)cut3/(float)cut2;
float EFFN4=(float)cut4/(float)cut3;
float EFFN5=(float)cut5/(float)cut4;


float EFFA1=(float)cut1/(float)numberOfEntries;
float EFFA2=(float)cut2/(float)numberOfEntries;
float EFFA3=(float)cut3/(float)numberOfEntries;
float EFFA4=(float)cut4/(float)numberOfEntries;
float EFFA5=(float)cut5/(float)numberOfEntries;

cout<<"EFFA1="<<EFFA1<<endl;
cout<<"EFFA2="<<EFFA2<<endl;
cout<<"EFFA3="<<EFFA3<<endl;
cout<<"EFFA4="<<EFFA4<<endl;
cout<<"EFFA5="<<EFFA5<<endl<<endl;


cout<<"EFFN1="<<EFFN1<<endl;
cout<<"EFFN2="<<EFFN2<<endl;
cout<<"EFFN3="<<EFFN3<<endl;
cout<<"EFFN4="<<EFFN4<<endl;
cout<<"EFFN5="<<EFFN5<<endl;

In [ ]:
/*Instancio un objeto TChain*/
TChain chainB1("Delphes");
/*Creo las variables necesarias para el for*/
std::string nomB1="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-DY_2j/Events/run_";
std::string nom2B1="/tag_1_delphes_events.root";
std::string numB1, outB1;
/*Llenado del chain con todos los runs*/
for ( i=1; i<3; i=i+1 ) {
   numB1 = std::to_string(i);
   if( i < 10 ) {
       outB1= nomB1+"0"+numB1+nom2B1;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * cB1 = outB1.c_str();
/*pego todos los datos*/
    chainB1.Add(cB1);
}
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReaderB1 = new ExRootTreeReader(&chainB1);
/*Leo el numero de entradas*/
Long64_t numberOfEntriesB1 = treeReaderB1->GetEntries();
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuonB1 = treeReaderB1->UseBranch("Muon");
TClonesArray *branchMissingETB1 = treeReaderB1->UseBranch("MissingET");
TClonesArray *branchJetB1 = treeReaderB1->UseBranch("Jet");
// Inicializo los contadores
Int_t cut1B1=0, cut2B1=0, cut3B1=0, cut4B1=0, cut5B1=0;
for(Int_t entryB1 = 0; entryB1 < numberOfEntriesB1; ++entryB1)
    {
        // Se carga un evento espesifico para ser analizado
        treeReaderB1->ReadEntry(entryB1);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJetB1->GetEntries() > 1)
        {
            ++cut1B1; //Cuantos eventos pasan el corte 1
            Int_t countB1=0;

            for (Int_t i=0; i<branchMuonB1->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJet->At(i);
                if(jet->BTag>0)
                {
                    ++countB1;
                }
    
            }

            if( countB1>1 )
            {
                ++cut2B1; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1B1, *muon2B1;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuonB1->GetEntries() > 1)
                {
                    ++cut3B1; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1B1 = (Muon *) branchMuonB1->At(0);
                    muon2B1 = (Muon *) branchMuonB1->At(1);
                    //Si la carga es diferente
                    int chargeB1 = muon1B1->Charge * muon2B1->Charge;
                    if( chargeB1 < 0 )
                    {
                        ++cut4B1; //Cuantos eventos pasan el corte 4
                
                        double MassMuonB1=(muon1B1->P4()+muon2B1->P4()).M();
                        if( !((MassMuonB1>80)&&(MassMuonB1<100)) )
                        {
                            ++cut5B1; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }
cout<<"#Raw ="<<numberOfEntriesB1<<endl;
cout<<"#Cut1="<<cut1B1<<endl;
cout<<"#Cut2="<<cut2B1<<endl;
cout<<"#Cut3="<<cut3B1<<endl;
cout<<"#Cut4="<<cut4B1<<endl;
cout<<"#Cut5="<<cut5B1<<endl<<endl;


float EFFN1B1=(float)cut1B1/(float)numberOfEntriesB1;
float EFFN2B1=(float)cut2B1/(float)cut1B1;
float EFFN3B1=(float)cut3B1/(float)cut2B1;
float EFFN4B1=(float)cut4B1/(float)cut3B1;
float EFFN5B1=(float)cut5B1/(float)cut4B1;


float EFFA1B1=(float)cut1B1/(float)numberOfEntriesB1;
float EFFA2B1=(float)cut2B1/(float)numberOfEntriesB1;
float EFFA3B1=(float)cut3B1/(float)numberOfEntriesB1;
float EFFA4B1=(float)cut4B1/(float)numberOfEntriesB1;
float EFFA5B1=(float)cut5B1/(float)numberOfEntriesB1;

cout<<"EFFA1="<<EFFA1B1<<endl;
cout<<"EFFA2="<<EFFA2B1<<endl;
cout<<"EFFA3="<<EFFA3B1<<endl;
cout<<"EFFA4="<<EFFA4B1<<endl;
cout<<"EFFA5="<<EFFA5B1<<endl<<endl;


cout<<"EFFN1="<<EFFN1B1<<endl;
cout<<"EFFN2="<<EFFN2B1<<endl;
cout<<"EFFN3="<<EFFN3B1<<endl;
cout<<"EFFN4="<<EFFN4B1<<endl;
cout<<"EFFN5="<<EFFN5B1<<endl;

In [ ]:
/*Instancio un objeto TChain*/
TChain chainB2("Delphes");
/*Creo las variables necesarias para el for*/
std::string nomB2="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-DY_2j/Events/run_";
std::string nom2B2="/tag_1_delphes_events.root";
std::string numB2, outB2;
/*Llenado del chain con todos los runs*/
for ( i=1; i<3; i=i+1 ) {
   numB1 = std::to_string(i);
   if( i < 10 ) {
       outB2= nomB2+"0"+numB2+nom2B2;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * cB2 = outB2.c_str();
/*pego todos los datos*/
    chainB2.Add(cB2);
}
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReaderB2 = new ExRootTreeReader(&chainB2);
/*Leo el numero de entradas*/
Long64_t numberOfEntriesB2 = treeReaderB2->GetEntries();
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuonB2 = treeReaderB2->UseBranch("Muon");
TClonesArray *branchMissingETB2 = treeReaderB2->UseBranch("MissingET");
TClonesArray *branchJetB2 = treeReaderB2->UseBranch("Jet");
// Inicializo los contadores
Int_t cut1B2=0, cut2B2=0, cut3B2=0, cut4B2=0, cut5B2=0;
for(Int_t entryB2 = 0; entryB2 < numberOfEntriesB2; ++entryB2)
    {
        // Se carga un evento espesifico para ser analizado
        treeReaderB2->ReadEntry(entryB2);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJetB2->GetEntries() > 1)
        {
            ++cut1B2; //Cuantos eventos pasan el corte 1
            Int_t countB2=0;

            for (Int_t i=0; i<branchMuonB2->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJetB2->At(i);
                if(jet->BTag>0)
                {
                    ++countB2;
                }
    
            }

            if( countB2>1 )
            {
                ++cut2B2; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1B2, *muon2B2;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuonB2->GetEntries() > 1)
                {
                    ++cut3B2; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1B2 = (Muon *) branchMuonB2->At(0);
                    muon2B2 = (Muon *) branchMuonB2->At(1);
                    //Si la carga es diferente
                    int chargeB2 = muon1B2->Charge * muon2B2->Charge;
                    if( chargeB2 < 0 )
                    {
                        ++cut4B2; //Cuantos eventos pasan el corte 4
                
                        double MassMuonB2=(muon1B2->P4()+muon2B2->P4()).M();
                        if( !((MassMuonB2>80)&&(MassMuonB2<100)) )
                        {
                            ++cut5B2; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }
cout<<"#Raw ="<<numberOfEntriesB2<<endl;
cout<<"#Cut1="<<cut1B2<<endl;
cout<<"#Cut2="<<cut2B2<<endl;
cout<<"#Cut3="<<cut3B2<<endl;
cout<<"#Cut4="<<cut4B2<<endl;
cout<<"#Cut5="<<cut5B2<<endl<<endl;


float EFFN1B2=(float)cut1B2/(float)numberOfEntriesB2;
float EFFN2B2=(float)cut2B2/(float)cut1B2;
float EFFN3B2=(float)cut3B2/(float)cut2B2;
float EFFN4B2=(float)cut4B2/(float)cut3B2;
float EFFN5B2=(float)cut5B2/(float)cut4B2;


float EFFA1B2=(float)cut1B2/(float)numberOfEntriesB2;
float EFFA2B2=(float)cut2B2/(float)numberOfEntriesB2;
float EFFA3B2=(float)cut3B2/(float)numberOfEntriesB2;
float EFFA4B2=(float)cut4B2/(float)numberOfEntriesB2;
float EFFA5B2=(float)cut5B2/(float)numberOfEntriesB2;

cout<<"EFFA1="<<EFFA1B2<<endl;
cout<<"EFFA2="<<EFFA2B2<<endl;
cout<<"EFFA3="<<EFFA3B2<<endl;
cout<<"EFFA4="<<EFFA4B2<<endl;
cout<<"EFFA5="<<EFFA5B2<<endl<<endl;


cout<<"EFFN1="<<EFFN1B2<<endl;
cout<<"EFFN2="<<EFFN2B2<<endl;
cout<<"EFFN3="<<EFFN3B2<<endl;
cout<<"EFFN4="<<EFFN4B2<<endl;
cout<<"EFFN5="<<EFFN5B2<<endl;

In [ ]:
/*Instancio un objeto TChain*/
TChain chainB3("Delphes");
/*Creo las variables necesarias para el for*/
std::string nomB3="/Storage/scratch/camilo/Proyectos/MadGraph/Datos/BackGround-DY_2j/Events/run_";
std::string nom2B3="/tag_1_delphes_events.root";
std::string numB3, outB3;
/*Llenado del chain con todos los runs*/
for ( i=1; i<3; i=i+1 ) {
   numB3 = std::to_string(i);
   if( i < 10 ) {
       outB3= nomB3+"0"+numB3+nom2B3;
    }
//convierto en cadena de caracteres constante (es lo que recibe la función)
    const char * cB3 = outB3.c_str();
/*pego todos los datos*/
    chainB3.Add(cB3);
}
//Creo un objeto de treereader, que recive la dirección en memoria de
//la cadena (TChain) como argumento
ExRootTreeReader *treeReaderB3 = new ExRootTreeReader(&chainB3);
/*Leo el numero de entradas*/
Long64_t numberOfEntriesB3 = treeReaderB3->GetEntries();
/*Obtengo las ramas que deseo*/
TClonesArray *branchMuonB3 = treeReaderB3->UseBranch("Muon");
TClonesArray *branchMissingETB3 = treeReaderB3->UseBranch("MissingET");
TClonesArray *branchJetB3 = treeReaderB3->UseBranch("Jet");
// Inicializo los contadores
Int_t cut1B3=0, cut2B3=0, cut3B3=0, cut4B3=0, cut5B3=0;
for(Int_t entryB3 = 0; entryB3 < numberOfEntriesB3; ++entryB3)
    {
        // Se carga un evento espesifico para ser analizado
        treeReaderB3->ReadEntry(entryB3);

        // Se hace el primer corte, se pide que tenga al menos dos jets
        if(branchJetB3->GetEntries() > 1)
        {
            ++cut1B3; //Cuantos eventos pasan el corte 1
            Int_t countB3=0;

            for (Int_t i=0; i<branchMuonB3->GetEntries(); ++i)
            {
                Jet* jet = (Jet*) branchJetB3->At(i);
                if(jet->BTag>0)
                {
                    ++countB3;
                }
    
            }

            if( countB3>1 )
            {
                ++cut2B3; //Cuantos eventos pasan el corte 2
                //Inicializo los obejos clase muon
                Muon *muon1B3, *muon2B3;

                // Para ser contados deben de tener al menos 2 muones
                if(branchMuonB3->GetEntries() > 1)
                {
                    ++cut3B3; //Cuantos eventos pasan el corte 3
                    // Take first two electrons, tomo solo dos porque estan organizados por PT
                    muon1B3 = (Muon *) branchMuonB3->At(0);
                    muon2B3 = (Muon *) branchMuonB3->At(1);
                    //Si la carga es diferente
                    int chargeB3 = muon1B3->Charge * muon2B3->Charge;
                    if( chargeB3 < 0 )
                    {
                        ++cut4B3; //Cuantos eventos pasan el corte 4
                
                        double MassMuonB3=(muon1B3->P4()+muon2B3->P4()).M();
                        if( !((MassMuonB3>80)&&(MassMuonB3<100)) )
                        {
                            ++cut5B3; //Cuantos eventos pasan el corte 4
                     
                            
                            // Plot jet transverse momentum
                            //histJetPT->Fill(jet->PT);
                            // Plot their invariant mass
                            //histMass->Fill(((elec1->P4()) + (elec2->P4())).M());
               
                        
                        
                        }
                    }
                }
            }
        }
    }
cout<<"#Raw ="<<numberOfEntriesB3<<endl;
cout<<"#Cut1="<<cut1B3<<endl;
cout<<"#Cut2="<<cut2B3<<endl;
cout<<"#Cut3="<<cut3B3<<endl;
cout<<"#Cut4="<<cut4B3<<endl;
cout<<"#Cut5="<<cut5B3<<endl<<endl;


float EFFN1B3=(float)cut1B3/(float)numberOfEntriesB3;
float EFFN2B3=(float)cut2B3/(float)cut1B3;
float EFFN3B3=(float)cut3B3/(float)cut2B3;
float EFFN4B3=(float)cut4B3/(float)cut3B3;
float EFFN5B3=(float)cut5B3/(float)cut4B3;


float EFFA1B3=(float)cut1B3/(float)numberOfEntriesB3;
float EFFA2B3=(float)cut2B3/(float)numberOfEntriesB3;
float EFFA3B3=(float)cut3B3/(float)numberOfEntriesB3;
float EFFA4B3=(float)cut4B3/(float)numberOfEntriesB3;
float EFFA5B3=(float)cut5B3/(float)numberOfEntriesB3;

cout<<"EFFA1="<<EFFA1B3<<endl;
cout<<"EFFA2="<<EFFA2B3<<endl;
cout<<"EFFA3="<<EFFA3B3<<endl;
cout<<"EFFA4="<<EFFA4B3<<endl;
cout<<"EFFA5="<<EFFA5B3<<endl<<endl;


cout<<"EFFN1="<<EFFN1B3<<endl;
cout<<"EFFN2="<<EFFN2B3<<endl;
cout<<"EFFN3="<<EFFN3B3<<endl;
cout<<"EFFN4="<<EFFN4B3<<endl;
cout<<"EFFN5="<<EFFN5B3<<endl;